In [1]:
import time
import requests
import pandas as pd

In [4]:
#Place API key
API_KEY= 'Place API'
url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

In [5]:
#list of areas of attractions
areas = pd.read_csv('area_Italy_Spain_Netherland.csv').fillna('').values.tolist()
categories = pd.read_csv('category.csv')['Category'].values.tolist()

In [6]:
def generate_query(category, area):
    unique_items = []
    for item in area:
        if item != '' and item not in unique_items:
            unique_items.append(item)
    unique_items.reverse()
    result = ', '.join(unique_items)
    return(f'{category}, {result}')

In [6]:
#Define function to send request and process data downloaded
def get_POI(API_KEY, category, area, pagetoken):
    query = generate_query(category, area)
    print(query)
    params = {
        'query': query,
        'key': API_KEY,
        'pagetoken' : pagetoken,
        }
    result = requests.get(url, params=params)
    #Convert data into json type first and dataframe then
    result_js = result.json()
    result_df = pd.json_normalize(result_js['results'])
    #Check if there is next page and send the token to params
    if 'next_page_token' in result_js:
        page_token = result_js['next_page_token']
    else:
        page_token = None
    return(result_df, page_token)

In [10]:
request_times = 0
page_token = None
POI_list = pd.DataFrame()

In [ ]:
#Main
for area in areas:
    for category in categories:
        print(f'request{request_times}:')
        result_df, page_token = get_POI(API_KEY, category, area, page_token)
        POI_list = pd.concat([POI_list,result_df])
        # Place API - 6000 qpm
        time.sleep(0.1)
        request_times += 1
        while page_token is not None:
                print(f'request{request_times}:')
                result_df, page_token = get_POI(API_KEY, category, area, page_token)
                result_df = pd.concat([POI_list,result_df])
                time.sleep(0.1)
                request_times += 1

In [ ]:
POI_list.drop_duplicates(subset=['place_id']).to_csv('POI_list.csv')